In [1]:
from __future__ import print_function

In [2]:
import numpy as np
import glob
from scipy import misc
import imageio
from keras.layers import *
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import Callback

Using TensorFlow backend.


In [3]:
imgs = glob.glob('../img_align_celeba/*.jpg')
np.random.shuffle(imgs)

In [4]:
height, width = imageio.imread(imgs[0]).shape[:2]
#cut square images
center_height = int((height - width) / 2)
img_dim = 64
z_dim = 512

In [5]:
def imread(f):
    x = imageio.imread(f)
    x = x[center_height:center_height + width, :]
    x = misc.imresize(x, (img_dim, img_dim))
    return x.astype(np.float32) / 255 * 2 - 1 #[-1, 1], tanh activation finally

In [23]:
def data_generator(batch_size=32):
    X = []
    while True:
        np.random.shuffle(imgs)
        for f in imgs:
            X.append(imread(f))
            if len(X) == batch_size:
                X = np.array(X)
                yield X,None
                X = []

In [7]:
x_in = Input(shape = (img_dim, img_dim, 3))
x = Conv2D(z_dim//16, kernel_size = (5,5), strides = (2,2), padding = 'SAME')(x_in)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)
x = Conv2D(z_dim//8, kernel_size = (5,5), strides = (2,2), padding = 'SAME')(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)
x = Conv2D(z_dim//4, kernel_size = (5,5), strides = (2,2), padding = 'SAME')(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)
x = Conv2D(z_dim//2, kernel_size = (5,5), strides = (2,2), padding = 'SAME')(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)
x = Conv2D(z_dim, kernel_size = (5,5), strides = (2,2), padding = 'SAME')(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)
x = GlobalAveragePooling2D()(x)

encoder = Model(x_in, x)
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        2432      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        51264     
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 64)        0         
__________

In [8]:
map_size = K.int_shape(encoder.layers[-2].output)[1:-1]

In [9]:
#decoder
z_in = Input(shape = K.int_shape(x)[1:])
z = Dense(np.prod(map_size) * z_dim)(z_in)
z = Reshape(map_size + (z_dim,))(z) # (2, 2, 512)
z = Conv2DTranspose(z_dim // 2, kernel_size = (5,5), strides = (2,2), padding = 'SAME')(z)
z = BatchNormalization()(z)
z = Activation('relu')(z)
z = Conv2DTranspose(z_dim // 4, kernel_size = (5,5), strides = (2,2), padding = 'SAME')(z)
z = BatchNormalization()(z)
z = Activation('relu')(z)
z = Conv2DTranspose(z_dim // 8, kernel_size = (5,5), strides = (2,2), padding = 'SAME')(z)
z = BatchNormalization()(z)
z = Activation('relu')(z)
z = Conv2DTranspose(z_dim // 16, kernel_size = (5,5), strides = (2,2), padding = 'SAME')(z)
z = BatchNormalization()(z)
z = Activation('relu')(z)
z = Conv2DTranspose(3, kernel_size = (5,5), strides = (2,2), padding = 'SAME')(z)
z = Activation('tanh')(z)

decoder = Model(z_in, z)
decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              1050624   
_________________________________________________________________
reshape_1 (Reshape)          (None, 2, 2, 512)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 4, 4, 256)         3277056   
_________________________________________________________________
batch_normalization_6 (Batch (None, 4, 4, 256)         1024      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 8, 8, 128)         819328    
__________

In [10]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape = (K.shape(z_mean)[0], z_dim), mean = 0., stddev = 1.)
    return z_mean + K.exp(z_log_var / 2) * epsilon

In [11]:
z_mean = Dense(z_dim)(x)
z_log_var = Dense(z_dim)(x)

z = Lambda(sampling, output_shape = (z_dim,))([z_mean, z_log_var])

In [12]:
z_mean.shape

TensorShape([Dimension(None), Dimension(512)])

In [17]:
x_recon = decoder(z)
x_out = Subtract()([x_in, x_recon])

recon_loss = 0.5 * K.sum(K.mean(x_out ** 2, 0))
kl_loss = -0.5 * K.mean(K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis = -1))

In [19]:
vae = Model(x_in, x_out)
vae.add_loss(recon_loss + kl_loss)
vae.compile(optimizer = Adam(1e-4))

In [21]:
#generate some images 
def sample(path):
    n = 9
    figure = np.zeros((img_dim * n, img_dim * n, 3))
    for i in range(n):
        for j in range(n):
            x_recon = decoder.predict(np.random.randn(1, *K.int_shape(x)[1:]))
            digit = x_recon[0]
            figure[i*img_dim:(i+1)*img_dim, j*img_dim:(j+1)*img_dim] = digit
    figure = (figure + 1) / 2 * 255
    imageio.imwrite(path, figure)

In [26]:
class Evaluate(Callback):
    def __init__(self):
        import os
        self.lowest = 1e10
        self.losses = []
        if not os.path.exists('samples'):
            os.mkdir('samples')
    
    def on_epoch_end(self, epoch, logs = None):
        path = 'samples/test_%s.png' % epoch
        sample(path)
        self.losses.append((epoch, logs['loss']))
        if logs['loss'] <= self.lowest:
            self.lowest = logs['loss']
            encoder.save_weights('./best_encoder.weights')

In [ ]:
evaluator = Evaluate()

vae.fit_generator(data_generator(),
                 epochs = 1000,
                 steps_per_epoch = 1000,
                 callbacks = [evaluator])

Epoch 1/1000


/home/niko/.conda/envs/tf_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  after removing the cwd from sys.path.


1000/1000 [==============================] - 388s 388ms/step - loss: 573.9605


Lossy conversion from float64 to uint8. Range [15.57491585612297, 238.34959745407104]. Convert image to uint8 prior to saving to suppress this warning.


Epoch 2/1000
1000/1000 [==============================] - 361s 361ms/step - loss: 509.8218


...

Epoch 104/1000
1000/1000 [==============================] - 70s 70ms/step - loss: 288.4974


Lossy conversion from float64 to uint8. Range [0.03577128052711487, 254.8218883574009]. Convert image to uint8 prior to saving to suppress this warning.


Epoch 105/1000
1000/1000 [==============================] - 73s 73ms/step - loss: 288.6116


Lossy conversion from float64 to uint8. Range [0.06563767790794373, 254.98331129550934]. Convert image to uint8 prior to saving to suppress this warning.


Epoch 106/1000
 632/1000 [=================>............] - ETA: 26s - loss: 287.9815